In [1]:
from tensorflow.keras.models import load_model
import numpy as np
from itertools import product
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.densenet import preprocess_input
import os
import warnings
warnings.filterwarnings('ignore')

2024-03-18 15:21:37.478714: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-18 15:21:37.478772: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-18 15:21:37.480343: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# Define constants

In [2]:
mappings = {'100000': ['a', '1'], '110000': ['b', '2'], '100100': ['c', '3'], 
            '100110': ['d', '4'], '100010': ['e', '5'], '110100': ['f', '6'], 
            '110110': ['g', '7'], '110010': ['h', '8'], '010100': ['i', '9'], 
            '010110': ['j', '0'], '101000': 'k', '111000': 'l', '101100': 'm', 
            '101110': 'n', '101010': 'o', '111100': 'p', '111110': 'q', 
            '111010': 'r', '011100': 's', '011110': 't', '101001': 'u',
            '111001': 'v', '010111': 'w', '101101': 'x', '101111': 'y',
            '101011': 'z', '000000': ' ', '011001': '?','011010': '!', 
            '001000': '\'', '010000': ',', '001001': '-', '010011': '.', 
            '011000': ';', '001111': '#', '000001': 'CAP'}

In [3]:
combinations = list(product([0, 1], repeat=6))
class_names = [''.join(map(str, comb)) for comb in combinations]
data_names = {}
predictions_arr_zeros_ones, predictions_arr = [], []
next_is_num, next_is_cap = False, False

In [4]:
data_path = '/kaggle/input/dataset-preprocessed/preprocessed_data.pkl'
model_path = '/kaggle/input/final-model-1/custom_data_model.h5'
test_data_path = '/kaggle/input/test-data-1'

# Sorting and preprocessing

In [5]:
# This step is done becaue kaggle messes up the order of
# the images in the working directory. This step ensures
# that the images provided to the model are in the same
# order in which they were obtained from the original
# scanned image.

for im in os.listdir(test_data_path):
    img_path = os.path.join(test_data_path, im)
    im_number = int(im.split('.')[0]) # will be used to sort the images dictionary
    img = image.load_img(img_path, target_size=(224, 224))  
    x = image.img_to_array(img)
    x = preprocess_input(x)
    x /= 255.0
    x = np.expand_dims(x, axis=0)
    
    data_names[im_number] = x

data_names_ordered = dict(sorted(data_names.items()))

# Load the model

In [6]:
model = load_model(model_path)

# Inference

In [ ]:
# Logic 1

for i, im_arr in enumerate(data_names_ordered.values()):
    predictions = model.predict(im_arr, verbose=0)
    predicted_class_index = np.argmax(predictions)
    predicted_class_label = class_names[predicted_class_index]
    predictions_arr_zeros_ones.append(predicted_class_label) 
    
    # Mapping encoded characters to their values in the mappings dict
    if predicted_class_label in mappings:
        predictions_arr.append(mappings[predicted_class_label])

In [ ]:
# Logic 2: better

for i, im_arr in enumerate(data_names_ordered.values()):
    predictions = model.predict(im_arr, verbose=0)
    predicted_class_index = np.argmax(predictions)
    predicted_class_label = class_names[predicted_class_index]
    predictions_arr_zeros_ones.append(predicted_class_label) 
    
    # Mapping encoded characters to their values in the mappings dict
    if predicted_class_label in mappings:
        if mappings[predicted_class_label] == '#': 
            next_is_num = True
        elif mappings[predicted_class_label] == 'CAP': 
            next_is_cap = True
        else:
            if next_is_num:
                predictions_arr.append(mappings[predicted_class_label][1])
                next_is_num = False
            elif next_is_cap:
                predictions_arr.append(mappings[predicted_class_label][0].upper())
                next_is_cap = False
            else:
                predictions_arr.append(mappings[predicted_class_label][0])

In [9]:
# Logic 3: Best

for im_arr in data_names_ordered.values():
    predictions = model.predict(im_arr, verbose=0)
    predicted_class_index = np.argmax(predictions)
    predicted_class_label = class_names[predicted_class_index]
    predictions_arr_zeros_ones.append(predicted_class_label) 
    
    # Mapping encoded characters to their values in the mappings dict
    if predicted_class_label in mappings:
        if mappings[predicted_class_label] == '#': 
            next_is_num = True
            num_str = ''
        elif mappings[predicted_class_label] == 'CAP': 
            next_is_cap = True
        else:
            if next_is_num:
                if mappings[predicted_class_label] != ' ' and isinstance(mappings[predicted_class_label], list):
                    num_str += mappings[predicted_class_label][1]
                else:
                    predictions_arr.append(num_str)
                    predictions_arr.append(mappings[predicted_class_label])
                    next_is_num = False
            elif next_is_cap:
                predictions_arr.append(mappings[predicted_class_label][0].upper())
                next_is_cap = False
            else:
                predictions_arr.append(mappings[predicted_class_label][0])

I0000 00:00:1710775346.360864   90277 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


# Result

In [10]:
print(''.join(predictions_arr))

                                5pErceptron, while the 1980s     witnessed setbacks due to         rimitations in computational      power. However, the late 1980s  and 1990s marked a significant   comeback with advancements in     algorithms like backpropagation   and the emergence of Support     Vector Machines. The 21st     century has witnessed a true      explosion in machine rearning,    fueled by the increasing          availability of data and computingpower. This has led to           breakthroughs in diverse fields   like image recognition, natural   language procesming, and robotics,shaping the world around us in    profound ways.                    
